In [19]:
import numpy as np
import matplotlib.pyplot as plt
from tensor.operation.kruskal import kruskal
from tensor.operation.khatri_rao import khatri_rao
from tensor.operation.matricize import matricize
from tensor.operation.sampled_khatri_rao_prod import SKR
from tensor.operation.sampled_matricize import Sampled_Matricize
import tensorly as tly
from tensorly.tenalg import mode_dot

Algorithm: CP RAND

$\text{Input:}$ $\mathcal{X} \in \mathbb{R}^{I_1 \times \cdots \times I_N}$, $\mathbf{R}$, $\mathbf{S}$

$\text{Output:}$ $\boldsymbol{\lambda}$, $\{\mathbf{A}^{(n)}\}$

$
\textbf{CPRAND} (\mathcal{X}, \mathbf{R}, \mathbf{S}):\\
\quad \text{Initialize factor matrices } \mathbf{A}^{(2)}, \dots, \mathbf{A}^{(N)}\\
\quad \textbf{repeat}:\\
\quad \quad \text{for } n = 1, \dots, N \text{ do}:\\
\quad \quad \quad \text{Define sampling operator} \mathbf{S} \in \mathbb{R}^{S \times \prod_{m \neq n} I_m}\\
\quad \quad \quad \mathbf{Z}_S \gets \text{SKR}(\mathbf{S}, \mathbf{A}^{(1)}, \dots, \mathbf{A}^{(n-1)}, \mathbf{A}^{(n+1)}, \dots, \mathbf{A}^{(N)})\\
\quad \quad \quad \mathbf{X}^T_S \gets \mathbf{S} \mathbf{X}^T_{(n)}\\
\quad \quad \quad \mathbf{A}^{(n)} \gets \underset{\mathbf{A}}{\arg \min} \lVert \mathbf{Z}_S \mathbf{A}^T - \mathbf{X}^T_S \rVert_F\\
\quad \quad \quad \text{Normalize columns of } \mathbf{A}^{(n)} \text{ and update } \boldsymbol{\lambda}\\
\quad \quad \textbf{end for}\\
\quad \textbf{until convergence}\\
\quad \text{Return } \boldsymbol{\lambda}, \{\mathbf{A}^{(n)}\}\\
$


In [20]:
class CP_rand_mix:
    def __init__(self, tensor: np.ndarray, rank :int, max_iter=10000, eps=0.01, initalisation:str ="random"):
        """
        PARAFAC decompositon, it initializes the maximum amount of iterations that 
        are done along with the tolerance value for the convergence
        it also takes the size of the core tensor rank
        """
        #Original Tensor
        self.tensor = tensor

        # mixed tensor
        self.mixed_tensor = None

        #Rank of decomposition being targeted
        self.rank = rank
        
        #Max iterations
        self.max_iter = max_iter

        #Stopping error value
        self.eps = eps

        #sets the type of initialisation
        self.init_type = initalisation

        #Stores the errrors at each step
        self.errors = []
        
        #Factor matrixes
        self.factors = []
        self.mixed_factors = [None for _ in range(self.tensor.ndim)]

        #Current lamda value
        self.lamda = np.ones(self.rank)

        #Diagnol matrices for sign-flip operation, diagnol entries = 1 or -1, size: (tensor.shape[i], tensor.shape[i])
        self.D = []
        for i in range(self.tensor.ndim):
            diag_signs = np.random.choice([-1, 1], size = self.tensor.shape[i])
            self.D.append(np.diag(diag_signs))

        self.F = []
        for i in range(self.tensor.ndim):
            self.F.append(np.random.rand(self.tensor.shape[i], self.tensor.shape[i]))

         
    def fit(self, check_convergence = True):
        self._init_factors()
        for i in range(self.max_iter):
            for mode in range(self.tensor.ndim):
                self._update_factors(mode)
            if check_convergence and self._is_converged():
                break
        print(f"Converged in {i+1} iterations")
    
    def _init_factors(self):
        """
        initialize the factors using the `rank` many left singular 
        vectors of the corresponding mode-n matricization of the input tensor
        """
        if self.init_type == 'random':
            self.factors = [np.random.rand(self.tensor.shape[i], self.rank) for i in range(self.tensor.ndim)]
        # elif self.init_type == 'hosvd':
        #     self.factors = []
        #     for i in range(self.tensor.ndim):
        #         M = np.linalg.svd(matricize(self.tensor, i))[0]
        #         if M.shape[1] < self.rank:
        #             M_ = np.zeros((M.shape[0], self.rank - M.shape[1]))
        #             M = np.concatenate((M, M_), axis=1)
        #         else:
        #             M = M[:, :self.rank]
        #         self.factors.append(M)
        else:
            raise Exception("Invalid initialisation method")
        
        # mix the factor matrices
        for i in range(self.tensor.ndim):
            self.mixed_factors[i] = self.F[i] @ self.D[i] @ self.factors[i]
            assert self.mixed_factors[i].shape == (self.tensor.shape[i], self.rank)

        # mix the tensor
        self.mixed_tensor = self.tensor
        for i in range(self.tensor.ndim):
            self.mixed_tensor = mode_dot(self.mixed_tensor, self.F[i] @ self.D[i], i)
        assert self.mixed_tensor.shape == self.tensor.shape
    
    def _update_factors(self, mode):
        """
        Update the factors per iteration for the (mode+1)th Factor
        """
        tot_row=1
        for x in range(self.tensor.ndim):
            if x!=mode:
                tot_row*=self.tensor.shape[x]
        S = 10 * self.rank

        sel_rows = np.random.choice(tot_row, S, replace=True)
        
        #This converges quite well
        # sel_rows=np.random.permutation(tot_row)

        Z_s_hat = SKR(sel_rows, self.mixed_factors, mode)
        assert Z_s_hat.shape == (S, self.rank)
        # complex conjugate of F[mode]
        F_conj = np.conj(self.F[mode])
        assert F_conj.shape == (self.tensor.shape[mode], self.tensor.shape[mode])
        X_s_hat =  self.D[mode] @ F_conj @ Sampled_Matricize(sel_rows, self.tensor, mode)
        # print(X_s.shape)
        # print(Z_s.shape)
        # print(len(sel_rows))
        # print(self.rank)
        self.factors[mode] = X_s_hat @ np.linalg.pinv(Z_s_hat.T)
        # print(self.factors[mode].shape)
        
        col_norms = np.linalg.norm(self.factors[mode], axis=0)

        self.factors[mode] = self.factors[mode] / col_norms # normalize the self.factors[mode]
        self.mixed_factors[mode] = self.F[mode] @ self.D[mode] @ self.factors[mode]
        
        self.lamda=col_norms
    
    def _is_converged(self):
        """
        check if the algorithm has converged
        by computing the Frobenius norm of the difference between the current tensor
        and the tensor obtained by multiplying the factors
        """
        tmp=self.factors[0]
        
        self.factors[0] =self.factors[0]* self.lamda
        estim = kruskal(*self.factors)
        error = np.linalg.norm(self.tensor - estim)
        print("Error = ", error)
        self.errors.append(error)
        self.factors[0]=tmp
        return error < self.eps
    
    def plot_errors(self):
        plt.plot(self.errors)
        plt.legend(["Errors"])
        plt.xlabel("Iterations")
        plt.ylabel("Frobenius norm")
        plt.show()

In [21]:
X = np.random.rand(2,3,4)
R = 3
tly_ans = tly.decomposition.parafac(X,rank=R)
X_tly = tly.cp_to_tensor(tly_ans)
print(f"Error is: {np.linalg.norm(X - X_tly)}")
solver = CP_rand_mix(X, R, int(1e3), 0.01, "random")
solver.fit()
# print(factors[0].shape, factors[1].shape, factors[2].shape)

Error is: 0.40738385217836715
Error =  10.983288046787367
Error =  7.0975256934303435
Error =  6.3215758299120255
Error =  7.822270723021141
Error =  9.52868905813538
Error =  6.298564793529524
Error =  9.146213015895984
Error =  8.309900997496868
Error =  7.653327454035091
Error =  8.824461542181476
Error =  8.444637035668004
Error =  15.348804258022554
Error =  6.104541977585204
Error =  11.681114374695092
Error =  8.942635344670464
Error =  6.744489298044161
Error =  10.661114870439773
Error =  8.70199751916237
Error =  9.473459309974412
Error =  5.634189091193516
Error =  11.36815379412208
Error =  5.415050045862889
Error =  7.961432438980632
Error =  6.418020033419319
Error =  8.401774119604566
Error =  9.885749813202573
Error =  8.436620006727493
Error =  7.744136315624639
Error =  6.906522752574912
Error =  8.294470477958304
Error =  8.97090340388035
Error =  7.71628204625236
Error =  10.036731007546363
Error =  8.303802469689229
Error =  7.151018763162406
Error =  10.3771920310